In [1]:
import cv2
import numpy as np 
from matplotlib import pyplot as plt
import glob
import os
import pickle
from sklearn.preprocessing import LabelEncoder
%matplotlib inline

In [2]:
def crop_img(img,gray):
    retval, thresh_gray = cv2.threshold(gray, 127, 255, 0)

    contours, hierarchy = cv2.findContours(thresh_gray,cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    # Find object with the biggest bounding box
    mx = (0,0,0,0)      # biggest bounding box so far
    mx_area = 0
    for cont in contours:
        x,y,w,h = cv2.boundingRect(cont)
        area = w*h
        if area > mx_area:
            mx = x,y,w,h
            mx_area = area
    x,y,w,h = mx

    # Output to files
    roi = cv2.rectangle(img,(x,y),(x+w,y+h),(200,0,0),2)
    crop_img = img[y:y+h,x:x+w]
    return crop_img

In [3]:
def mask(img_crop, segmentation):
    img_contours = cv2.findContours(segmentation, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[-2]
    img_contours = sorted(img_contours, key=cv2.contourArea)
    for i in img_contours:
        if cv2.contourArea(i) > 100:
            break
    mask = np.zeros(img_crop.shape[:2], np.uint8)
    x = cv2.drawContours(mask, [i],-1, 255, -1)
    result = cv2.bitwise_and(img_crop,img_crop,mask = mask)
    return result

In [4]:
def segmentationL(img_crop):
    lab = cv2.cvtColor(img_crop, cv2.COLOR_BGR2Lab)
    median = cv2.medianBlur(lab,5)
    ret3,th3 = cv2.threshold(median[:,:,0],0,255,cv2.THRESH_OTSU)
    kernel = np.ones((19,19), np.uint8)
    closing = cv2.morphologyEx(th3, cv2.MORPH_CLOSE, kernel)
    opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel)
    img_dilation = cv2.dilate(opening, kernel, iterations=1) 
    return img_dilation

In [5]:
def segmentationB(img_crop):
    lab = cv2.cvtColor(img_crop, cv2.COLOR_BGR2Lab)
    median = cv2.medianBlur(lab,5)
    ret3,th3 = cv2.threshold(median[:,:,2],0,255,cv2.THRESH_OTSU)
    kernel = np.ones((19,19), np.uint8) 
    closing = cv2.morphologyEx(th3, cv2.MORPH_CLOSE, kernel)
    opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel) 
    img_dilation = cv2.dilate(opening, kernel, iterations=1) 
    return img_dilation

In [6]:
def show(img):
    img2 = img[:,:,::-1]
    plt.imshow(img2)

In [7]:
def featureExtractionHSV(img_segmentation):
    feature_matrix = np.zeros((250,300))
    img_hsv = cv2.cvtColor(img_segmentation, cv2.COLOR_BGR2HSV)
    for i in range(img_hsv.shape[0]):
        for j in range(img_hsv.shape[1]):
            feature_matrix[i][j] = (int(img_hsv[i,j,0]) + int(img_hsv[i,j,1]) + int(img_hsv[i,j,2]))/3
    feature_hsv = np.reshape(feature_matrix, ((250 * 300)))
    return feature_hsv
    

In [8]:
def featureExtractionRGB(img_segmentation):
    feature_matrix = np.zeros((250,300))
    for i in range(img_segmentation.shape[0]):
        for j in range(img_segmentation.shape[1]):
            feature_matrix[i][j] = (int(img_segmentation[i,j,0]) + int(img_segmentation[i,j,1]) + int(img_segmentation[i,j,2]))/3
    feature_rgb = np.reshape(feature_matrix, ((250 * 300)))
    return feature_rgb
    

In [9]:
def save_data(data, file_name):
    with open(file_name, 'wb') as file:
        pickle.dump(data, file)

In [19]:
path = "Segmentation Result/Buttercup"
dirs = os.listdir(path)
for file in dirs:
    img = cv2.imread("Data/Buttercup/"+file)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_crop = crop_img(img, gray)
    segmentation = segmentationB(img_crop)
    segmentation_crop = mask(img_crop, segmentation)
    img_resize = cv2.resize(segmentation_crop, (300,250))
    cv2.imwrite(os.path.join(path ,file), img_resize)

In [20]:
path = "Segmentation Result/Daisy"
dirs = os.listdir(path)
for file in dirs:
    img = cv2.imread("Data/Daisy/"+file)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_crop = crop_img(img, gray)
    segmentation = segmentationL(img_crop)
    segmentation_crop = mask(img_crop, segmentation)
    img_resize = cv2.resize(segmentation_crop, (300,250))
    cv2.imwrite(os.path.join(path ,file), img_resize)

In [21]:
path = "Segmentation Result/Dandelion"
dirs = os.listdir(path)
for file in dirs:
    img = cv2.imread("Data/Dandelion/"+file)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_crop = crop_img(img, gray)
    segmentation = segmentationB(img_crop)
    segmentation_crop = mask(img_crop, segmentation)
    img_resize = cv2.resize(segmentation_crop, (300,250))
    cv2.imwrite(os.path.join(path ,file), img_resize)

In [22]:
path = "Segmentation Result/Coltsfoot"
dirs = os.listdir(path)
for file in dirs:
    img = cv2.imread("Data/Coltsfoot/"+file)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_crop = crop_img(img, gray)
    segmentation = segmentationB(img_crop)
    segmentation_crop = mask(img_crop, segmentation)
    img_resize = cv2.resize(segmentation_crop, (300,250))
    cv2.imwrite(os.path.join(path ,file), img_resize)

In [23]:
path = "Segmentation Result/Windflower"
dirs = os.listdir(path)
for file in dirs:
    img = cv2.imread("Data/Windflower/"+file)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_crop = crop_img(img, gray)
    segmentation = segmentationL(img_crop)
    segmentation_crop = mask(img_crop, segmentation)
    img_resize = cv2.resize(segmentation_crop, (300,250))
    cv2.imwrite(os.path.join(path ,file), img_resize)